In [10]:
import requests
import pandas as pd
import numpy as np
from datetime import timedelta

# Pulling ETF Data

In [4]:
splits = pd.read_excel('etf_corp_actions.xlsx', sheet_name = 'SPLITS')
def adjust_splits(spy,ticker):
    test = spy.copy()
    if ticker in splits['symbol'].unique():
        splits_df = splits[splits['symbol']==ticker].sort_values(by = 'split_date').copy()
        splits_df['raw_adj_split'] = splits_df.groupby('symbol')['split'].cumprod()

        adjustments = []
        for x in range(len(splits_df)):

            start_date = splits_df.iloc[x,1]

            if x+1 == len(splits_df):
                end_date = test['DATE'].iloc[-1] + timedelta(days = 1)
            else:
                end_date = splits_df.iloc[x+1,1]
            split_ratio = splits_df.iloc[x,3]
            updated = test[(test['DATE']>=start_date)&(test['DATE']<end_date)].drop(['DATE','last_vol'], axis = 1)
            updated_col = updated.columns
            updated = updated/split_ratio
            adjustments.append(updated)
        adj_df = pd.concat(adjustments)
        # Update the original dataframe
        test.loc[adj_df.index, updated_col] = adj_df
    return test

In [ ]:
tickers = ['SPY','IJH','IWM','QQQ','DIA',
            'EWJ','EWY','EWC','EWU','EWA','EWG','EWZ',
           'GLD','SLV',
           'DBE','DBA','DBB']
data_dict = {}

for symbol in tickers:
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&outputsize=full&apikey={your_AV_key}"
    data = pd.DataFrame(requests.get(url).json()['Time Series (Daily)']).T.sort_index().reset_index()
    data = data.rename(columns={'index': 'DATE', '4. close':'last_close',
                                '1. open':'last_open', '2. high':'last_max','3. low':'last_min',
                                '5. volume':'last_vol'})
    data['DATE'] = pd.to_datetime(data['DATE'])
    data[data.columns[1:]] = data[data.columns[1:]].astype(float)
    #data.loc[:,'TICKER'] = symbol
    data = adjust_splits(data,symbol)
    data.to_pickle(symbol+'_daily.pkl')
    data_dict[symbol] = data


# Preparing Data

## Price Classifications

In [81]:
def get_price_class(df):
    df['CLASS'] = np.where((df['last_max']>df['last_max'].shift(1))&(df['last_min']>df['last_min'].shift(1)),
                           np.where(df['last_close']>df['last_open'],'U_UP','D_UP'),
                np.where((df['last_max']<df['last_max'].shift(1))&(df['last_min']<df['last_min'].shift(1)),
                         np.where(df['last_close']<df['last_open'],'D_DOWN','U_DOWN'),
                np.where(((df['last_max']>=df['last_max'].shift(1))&(df['last_min']<=df['last_min'].shift(1))),
                            np.where(df['last_close']>=df['last_open'], 'U_OUT','D_OUT'),
                np.where((df['last_max']<=df['last_max'].shift(1))&(df['last_min']>=df['last_min'].shift(1)),
                         np.where(df['last_close']>=df['last_open'], 'U_IN','D_IN'),'CHECK'))))
    df['LABEL'] = df['CLASS'].shift(-1)

## Feature Engineering Lag Features
5 Day, 10 Day, 20 Day, 40 Day, 60 Day, and 120 Day:
- ROC (OHLC)
- % to N-day max/min
- Coefficient of Variation of Returns


In [83]:
def get_lag_features(df):
    for lb in lookbacks:
        # OHLC ROC
        df['ROC_CLOSE_'+str(lb)]=(df['last_close']/df['last_close'].shift(lb))-1
        df['ROC_OPEN_'+str(lb)]=(df['last_open']/df['last_open'].shift(lb))-1
        df['ROC_HIGH_'+str(lb)]=(df['last_max']/df['last_max'].shift(lb))-1
        df['ROC_LOW_'+str(lb)]=(df['last_min']/df['last_min'].shift(lb))-1

        # OHLC % to MIN/MAX
        df['CLOSE_MIN_'+str(lb)] = (df['last_close'] / df.rolling(lb)['last_close'].min().shift(1))-1
        df['CLOSE_MAX_'+str(lb)] = (df['last_close'] / df.rolling(lb)['last_close'].max().shift(1))-1
        df['OPEN_MIN_'+str(lb)] = (df['last_open'] / df.rolling(lb)['last_open'].min().shift(1))-1
        df['OPEN_MAX_'+str(lb)] = (df['last_open'] / df.rolling(lb)['last_open'].max().shift(1))-1
        df['LOW_MIN_'+str(lb)] = (df['last_min'] / df.rolling(lb)['last_min'].min().shift(1))-1
        df['LOW_MAX_'+str(lb)] = (df['last_min'] / df.rolling(lb)['last_min'].max().shift(1))-1
        df['HIGH_MIN_'+str(lb)] = (df['last_max'] / df.rolling(lb)['last_max'].min().shift(1))-1
        df['HIGH_MAX_'+str(lb)] = (df['last_max'] / df.rolling(lb)['last_max'].max().shift(1))-1

        # CV
        df['CV_'+str(lb)] = df.rolling(lb)['last_close'].std()/df.rolling(lb)['last_close'].mean()

## Applying Feature Engineering

In [87]:
tickers = ['SPY','IJH','IWM','QQQ','DIA',
            'EWJ','EWY','EWC','EWU','EWA','EWG','EWZ',
           'GLD','SLV',
           'DBE','DBA','DBB']
data_dict = {}

for symbol in tickers:
    data = pd.read_pickle(symbol+'_daily.pkl')
    data_dict[symbol] = data

In [88]:
lookbacks = [5,10,20,40,60,120]
label_mapping = {'U_UP': 0, 'D_UP': 1, 'D_DOWN': 2, 'U_DOWN': 3, 'U_OUT': 4, 'D_OUT': 5, 'U_IN': 6, 'D_IN': 7}

In [89]:
for symbol in tickers:
    get_price_class(data_dict[symbol])
    get_lag_features(data_dict[symbol])

    if data_dict[symbol].loc[0,'CLASS'] != 'CHECK':
        print('check: '+symbol)
    else:
        # Remove CHECK (First Row)
        data_dict[symbol] = data_dict[symbol].iloc[1:]
        data_dict[symbol] = pd.get_dummies(data_dict[symbol],columns = ['CLASS'], drop_first= True)
        data_dict[symbol] = data_dict[symbol].drop(['DATE','last_open','last_max','last_close','last_min','last_vol'], axis = 1).dropna()
        data_dict[symbol]['LABEL'] = data_dict[symbol]['LABEL'].map(label_mapping)

## Creating Baskets

In [92]:
us_equity = ['SPY', 'IJH', 'QQQ', 'DIA', 'IWM']
intl_equity = ['EWJ', 'EWY', 'EWC', 'EWA', 'EWG', 'EWZ']
prec_metal = ['GLD','SLV']
comm = ['DBA','DBE','DBB']
use_df = pd.concat([data_dict[symbol] for symbol in us_equity], ignore_index=True)
ine_df = pd.concat([data_dict[symbol] for symbol in intl_equity], ignore_index=True)
pm_df = pd.concat([data_dict[symbol] for symbol in prec_metal], ignore_index=True)
com_df = pd.concat([data_dict[symbol] for symbol in comm], ignore_index=True)
sp_df = data_dict['SPY'] 

# Export Data for ML

In [93]:
use_df.to_pickle('US_EQUITY.pkl')
ine_df.to_pickle('INTL_EQUITY.pkl')
pm_df.to_pickle('PREC_METAL.pkl')
com_df.to_pickle('COMM.pkl')
sp_df.to_pickle('SPY.pkl')